This is a slightly more evolved script for processing data than `processing_explore.ipynb`. 
Will be converted into a proper python script later.

In [2]:
import os
from lisa.config import PILOT_DATA_DIR
import kineticstoolkit as ktk
import polars as pl
import warnings

# Suppress ktk timeseries warning
warnings.filterwarnings("ignore", category=UserWarning)

2024-07-18 17:01:25.804 | INFO     | lisa.config:<module>:13 - PROJ_ROOT path is: /Users/tomwilson/code/LISA


In [6]:
# Functions

def c3d_to_df(c3d_contents, type='Points'):
    pd_df = c3d_contents[type].to_dataframe()
    return pl.from_pandas(pd_df, include_index=True)

def process_time(df):
    # Remove t dim in points data
    columns_to_drop = [col for col in df.columns if col.endswith('[3]')]

    # Drop these columns from the DataFrame
    df = df.drop(columns_to_drop)

    # Name time column
    return df.rename(lambda col: "TIME" if "None" in col else col)

def find_activity_category(filename):
    for activity in activity_categories:
        if filename.lower().startswith(activity):
            return activity
    return None  # Return None or a default value if no match is found

def drop_stationary_markers(df):
    # Remove floor markers, defined as markers labelled with '*' and no null values

    # Get the count of the 'TIME' column
    time_count = df['TIME'].count()

    # Find columns that start with '*' and have the same count as the 'TIME' column
    matching_columns = [col for col in df.columns if col.startswith('*') and df[col].count() == time_count]

    print("Dropping columns: " + ', '.join(matching_columns))

    # Remove the identified columns from the DataFrame
    return df.drop(matching_columns)

In [9]:
activity_categories = ['walk', 'jog', 'run', 'jump']
total_df = None

# Ignore any non-c3d files or files that don't start with the activity categories, i.e. calibration files
for filename in os.listdir(PILOT_DATA_DIR):
    if filename.endswith(".c3d") and any(filename.lower().startswith(activity) for activity in activity_categories):
        print(filename)
        file = os.path.join(PILOT_DATA_DIR, filename)
        # TODO check convert_point_unit doesn't affect analog data
        c3d_contents = ktk.read_c3d(file, convert_point_unit=True)
        dfs = {}

        # TODO raise warning for missing data type
        for key in c3d_contents.keys():
            df = c3d_to_df(c3d_contents, key)
            if df.is_empty():
                break
            else:
                dfs[key] = df
            dfs[key] = process_time(dfs[key])
            dfs[key] = drop_stationary_markers(dfs[key])

        print(dfs.keys())
        if len(dfs) == 1:
            joined_df = next(iter(dfs.values()))
        else:
            joined_df = dfs['Points'].join(dfs['Analogs'], on='TIME')
            
        joined_df = joined_df.with_columns(pl.lit(find_activity_category(filename)).alias('ACTIVITY'))

        if total_df is None:
            total_df = joined_df
        else:
            total_df = pl.concat([total_df, joined_df], how='diagonal')

Walking 1_4ms01.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *38[0], *38[1], *38[2], *39[0], *39[1], *39[2]
Dropping columns: 
dict_keys(['Points', 'Analogs'])
Walk1_4ms 10incline01.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *37[0], *37[1], *37[2], *39[0], *39[1], *39[2]
Dropping columns: 
dict_keys(['Points', 'Analogs'])
Jogging 2_2.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *37[0], *37[1], *37[2], *39[0], *39[1], *39[2]
Dropping columns: 
dict_keys(['Points', 'Analogs'])
Walk1_4ms 5incline01.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *36[0], *36[1], *36[2], *37[0], *37[1], *37[2]
Dropping columns: 
dict_keys(['Points', 'Analogs'])
Walking 1_4ms02.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *37[0], *37[1], *37[2], *38[0], *38[1], *38[2]
dict_keys(['Points'])
Jogging 2_5ms01.c3d
Dropping columns: *34[0], *34[1], *34[2], *35[0], *35[1], *35[2], *38[0], *38[

KeyError: 'Points'

In [ ]:
# TODO move inside loop
filename = 'Walk1_7ms 5decline02'
file = os.path.join(PILOT_DATA_DIR, filename + '.c3d')

In [ ]:
# TODO check convert_point_unit doesn't affect analog data
c3d_contents = ktk.read_c3d(file, convert_point_unit=True)

In [ ]:
point_df = c3d_to_df(c3d_contents, type='Points')

point_df = process_time(point_df)

In [ ]:
# Remove floor markers

# Get the count of the 'TIME' column
time_count = point_df['TIME'].count()

# Find columns that start with '*' and have the same count as the 'TIME' column
matching_columns = [col for col in point_df.columns if col.startswith('*') and point_df[col].count() == time_count]

# Remove the identified columns from the DataFrame
point_df = point_df.drop(matching_columns)

print("Dropping columns: " + ', '.join(matching_columns))

Dropping columns: *35[0], *35[1], *35[2], *36[0], *36[1], *36[2], *38[0], *38[1], *38[2], *39[0], *39[1], *39[2]


In [ ]:
# Analog Data
analog_df = c3d_to_df(c3d_contents, 'Analogs')

analog_df = process_time(analog_df)

In [ ]:
joined_df = point_df.join(analog_df, on='TIME')

joined_df = joined_df.with_columns(pl.lit(find_activity_category(filename)).alias('ACTIVITY'))

In [ ]:
joined_df

TIME,RASIS[0],RASIS[1],RASIS[2],LPSIS[0],LPSIS[1],LPSIS[2],RPSIS[0],RPSIS[1],RPSIS[2],LLKN[0],LLKN[1],LLKN[2],LMKN[0],LMKN[1],LMKN[2],LT4[0],LT4[1],LT4[2],LT1[0],LT1[1],LT1[2],LT2[0],LT2[1],LT2[2],LT3[0],LT3[1],LT3[2],LLM[0],LLM[1],LLM[2],LMM[0],LMM[1],LMM[2],LSK4[0],LSK4[1],LSK4[2],…,*41[2],*42[0],*42[1],*42[2],Force.Fx1,Force.Fy1,Force.Fz1,Moment.Mx1,Moment.My1,Moment.Mz1,Force.Fx2,Force.Fy2,Force.Fz2,Moment.Mx2,Moment.My2,Moment.Mz2,Angle.Pitch,Length.Sway,Velocity.Left Belt Speed,Velocity.Right Belt Speed,Electric Potential.1,Global Angle.x,Global Angle.y,Global Angle.z,HighG.x,HighG.y,HighG.z,accel.x,accel.y,accel.z,gyro.x,gyro.y,gyro.z,mag.x,mag.y,mag.z,ACTIVITY
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0.0,0.11687,0.052211,1.011285,-0.058818,-0.124478,1.043918,0.046019,-0.123357,1.020504,-0.169058,-0.170044,0.521349,-0.046874,-0.149174,0.485763,-0.198806,-0.06316,0.700852,-0.199652,-0.102817,0.717995,-0.194537,-0.125516,0.679364,-0.193945,-0.082802,0.659023,-0.13262,-0.353778,0.135608,-0.046265,-0.331493,0.13522,-0.178658,-0.220675,0.390667,…,null,null,null,null,6.528756,-91.64798,2.936042,-4109.524902,6912.517578,24668.763672,-48.083618,-93.193214,-527.192444,101829.046875,67925.90625,-37220.324219,0.087164,-0.002245,-1.701906,-1.710057,1.840266,-0.823135,-1.447943,-0.270271,11.95933,13.394449,4.305358,10.448297,7.469909,-0.368707,1.20345,-0.120345,1.89783,-0.000037,-0.000032,-0.000018,"""walk"""
0.005,0.116933,0.053743,1.009175,-0.057159,-0.124106,1.042485,0.047901,-0.123199,1.018471,-0.168631,-0.175453,0.521933,-0.046739,-0.154131,0.485563,-0.198649,-0.066872,0.700477,-0.199163,-0.106379,0.717992,-0.193917,-0.129449,0.679623,-0.19368,-0.086905,0.658816,-0.13239,-0.362851,0.137685,-0.046146,-0.34038,0.137049,-0.178374,-0.227388,0.391691,…,null,null,null,null,1.115396,-92.93515,-5.776045,-18577.65625,-3299.269775,27846.941406,-51.637466,-65.968704,-556.365112,105257.117188,72322.460938,-28604.482422,0.087175,0.000502,-1.703584,-1.710515,1.816767,-0.82814,-1.443,-0.256443,12.916076,12.676889,3.707392,11.004949,6.749254,-0.393846,1.085235,-0.132592,2.075152,-0.000037,-0.000033,-0.000017,"""walk"""
0.01,0.116971,0.055333,1.007031,-0.055477,-0.123684,1.040882,0.049539,-0.121951,1.016533,-0.168144,-0.180602,0.52242,-0.046573,-0.15885,0.485311,-0.198463,-0.070363,0.699866,-0.198985,-0.109617,0.717924,-0.193311,-0.133109,0.679699,-0.193386,-0.090784,0.658484,-0.132129,-0.371926,0.139835,-0.045835,-0.349312,0.138918,-0.178012,-0.233805,0.392636,…,null,null,null,null,-4.618952,-116.083069,1.594693,-19034.611328,-4402.008789,34305.824219,-46.579582,-53.796627,-533.958923,118719.671875,67957.109375,-18018.537109,0.087164,-0.001635,-1.701753,-1.708379,1.809748,-0.833109,-1.438012,-0.242622,13.872822,11.95933,3.109426,11.5616,6.0286,-0.418985,0.96702,-0.14484,2.252475,-0.000037,-0.000034,-0.000016,"""walk"""
0.015,0.116971,0.056952,1.004813,-0.053826,-0.123271,1.039191,0.051025,-0.119986,1.014592,-0.16775,-0.185375,0.522685,-0.046381,-0.163326,0.485026,-0.198234,-0.073594,0.699116,-0.19857,-0.112617,0.717619,-0.19271,-0.13648,0.679616,-0.193069,-0.09434,0.658008,-0.131832,-0.380933,0.142148,-0.045543,-0.358138,0.140834,-0.177182,-0.240178,0.393149,…,null,null,null,null,-11.199624,-136.352997,-7.225827,-19388.96875,142.827026,45576.074219,-52.147915,-64.919113,-547.95282,123307.953125,65428.960938,-28816.347656,0.087175,-0.001024,-1.704042,-1.710057,1.800593,-0.838044,-1.432978,-0.228808,14.829569,11.24177,2.511459,12.118252,5.307945,-0.444124,0.848805,-0.157087,2.429797,-0.000037,-0.000034,-0.000015,"""walk"""
0.02,0.116922,0.05862,1.002475,-0.052127,-0.122788,1.037356,0.052698,-0.119417,1.012415,-0.167307,-0.189817,0.522904,-0.046197,-0.167603,0.484684,-0.198275,-0.076514,0.69776